# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['AngleX', 'AngleY', 'LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [263]:
%pylab inline
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics

ROBOT_POSE_DATA_DIR = 'robot_pose_data'

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [264]:
classes = listdir(ROBOT_POSE_DATA_DIR)
print classes

['Stand', 'Left', 'Crouch', 'Sit', 'Belly', 'Knee', 'StandInit', 'Back', 'Right', 'Frog', 'HeadBack']


In [265]:
def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    # YOUR CODE HERE
    
    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    data = pickle.load(open(filename))
    target = [i] * len(data)
    print len(data[0])
    return data, target

In [266]:
# load all the data
all_data = []
all_target = []

for i in range(0,len(classes)):
    data,target = load_pose_data(i)
    all_data.extend(data)
    all_target.extend(target)
    
    
print 'total number of data', len(all_data)

10
10
10
10
10
10
10
10
10
10
10
total number of data 200


## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [267]:
# shuffule data
permutation = np.random.permutation(len(all_data))
n_training_data = int(len(all_data) * 0.99)
training_data = permutation[:n_training_data]

In [268]:
print training_data

[ 35 191 164  65   2 119  74 124 155 109 145  73  30 112 149  90  29 159
  87  86  17  18 126 115 189 116 167 142  24 188  94 174  85 151 108  56
 182  53 179 170  42 154  20 137  60  71  47 181 122  89  25   5 160  80
  33 146 171 102 150  23  61  63 161 157 113 180  51 148  45 136  21 133
  97  76  70 101   8  84 168  72  12  66 175 184 186  31  78  79  62 162
 103 131 178  99  34  26 121  43 192 140  38 187  77  82 134  55 128  15
 117 110  40  67 143 196  44 100  50 138 190  41  16 147  75  32  57 114
  83 106  11  69 172 169  39  96 123 129   4 183  93  28  10  48  92   9
 135   1  54  19 130 193  13 111 185 132  52  58   3  27   6 153 107 163
  88 156 158 152 176 165 195 173 177 198 199  95  46  59  98 166  37 197
  64   7  91 105  36 144  81 139 194   0  68 104  14 125  22 141 118 127]


In [269]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [270]:
fit_data = []
fit_target = []
for i in range(0,len(training_data)):
    fit_data.append(all_data[training_data[i]])
    fit_target.append(all_target[training_data[i]])
    
clf.fit(fit_data, fit_target) 

SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0, degree=3,
  gamma=0.001, kernel='rbf', max_iter=-1, probability=False,
  random_state=None, shrinking=True, tol=0.001, verbose=False)

### predicting

In [271]:
clf.predict(all_data[1]), all_target[1]

(array([0]), 0)

In [272]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

## 4. Evaluate on the test data

In [273]:
expected = []
predicted = []
prediction_data = []
for i in range(0,len(all_data)):
    if( not (i in training_data)):
        expected.append(all_target[i])
        prediction_data.append(all_data[i])
predicted = clf.predict(prediction_data)

evaluate(expected, predicted)

Classification report:
             precision    recall  f1-score   support

          2       1.00      1.00      1.00         1
          6       1.00      1.00      1.00         1

avg / total       1.00      1.00      1.00         2


Confusion matrix:
[[1 0]
 [0 1]]


## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [274]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'w'))

Then, in the application we can load the trained classifier again.

In [275]:
clf2 = pickle.load(open(ROBOT_POSE_CLF))
clf2.predict(all_data[1]), all_target[1]

(array([0]), 0)